In [ ]:
import os
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import ElasticsearchStore

#extract question from csv
csv = []
#put csv question to query
query = ""

def neo4j_demo(query):
    loader = TextLoader("/Users/chrisbooth/Coding/ragvsgrag/MOD_IPR_Policy_Statement.txt")

    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    docs = text_splitter.split_documents(documents)

    # The Neo4jVector Module will connect to Neo4j and create a vector index if needed.

    db = Neo4jVector.from_documents(
        docs, embedding = OpenAIEmbeddings(), 
        url=os.getenv('NEO4J_URL'), 
        username=os.getenv('NEO4J_USERNAME'), 
        password=os.getenv('NEO4J_PASSWORD'),
    )

    query = "How does the MOD view the exploitation of IP for the wider UK economy?"
    docs_with_score = db.similarity_search_with_score(query, k=2)

    return docs_with_score

# Create neo4j with Michaels new search

def elastic(query):
    embeddings = OpenAIEmbeddings()

    loader = TextLoader("/Users/chrisbooth/Coding/ragvsgrag/MOD_IPR_Policy_Statement.txt") #change to web scraper 
    documents = loader.load()
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
    docs = text_splitter.split_documents(documents)

    vectorstore = ElasticsearchStore.from_documents(
        docs,
        embedding=embeddings,
        es_url="http://localhost:9200",
        es_user="elastic",
        es_password="cAe4zmUZlFg9+a78O4=F",
        index_name="test-basic",
    )

    vectorstore.client.indices.refresh(index="test-basic")

    elastic_results = vectorstore.similarity_search(query)
    
    return elastic_results


In [ ]:
#return answer and write info from each function of question back to csv


#Generate a QA pair for comparison and scoring?
# (Vineet said theyd feedback, grab that?)
